# Setup

* conda install rpy2
* conda install -c r r
* conda install -c r r-forecast

Kasulikku:

 * https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average
 * https://sites.google.com/site/aslugsguidetopython/data-analysis/pandas/calling-r-from-python
 * https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/
 * https://www.cheatography.com/weidadeyue/cheat-sheets/jupyter-notebook/

In [9]:
import numpy as np
import pandas as pd
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
numpy2ri.activate()

import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = "jpg"
plt.style.use("ggplot")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
from load_data import load_all_data_pd
from os.path import join
import numpy as np
import pandas as pd
import time
from evaluation import sMAPE, MASE, owa_10k, OWA
from collections import defaultdict
season_horizon = {"daily" : (1, 14),
                  "hourly": (24, 48),
                  "monthly": (12, 18),
                  "quarterly": (4, 8), 
                  "weekly": (1, 13),
                  "yearly": (1, 6)}

In [91]:
path_train = join("data","full","train")
path_test = join("data","full","test")
path_naive = join("naive_results", "naive1_10k_preds.csv")
#files = ["Daily.csv", "Hourly.csv", "Monthly.csv", "Quarterly.csv", "Weekly.csv", "Yearly.csv"]
#(train_da, test_da), (train_ho, test_ho), (train_mo, test_mo),\
#(train_qu, test_qu), (train_we, test_we), (train_ye, test_ye) = load_all_data_pd(path_train, path_test, files)
#all_data = {"daily" : (train_da, test_da),
#            "hourly": (train_ho, test_ho),
#            "monthly": (train_mo, test_mo),
#            "quarterly": (train_qu, test_qu), 
#            "weekly": (train_we, test_we),
#            "yearly": (train_ye, test_ye)}

files = ["Monthly.csv"]
mo = load_all_data_pd(path_train, path_test, files)
train_mo = mo[0][0]
test_mo = mo[0][1]
all_data = {"monthly": (train_mo, test_mo)}

In [18]:
forecast = importr('forecast')

In [20]:
# for final forecasting
def transform_2(data,transformation,na_action=False):
    dat = np.array(data)
    lmbda = boxcox(dat)[1]
    if transformation=='boxcox':
        dat = boxcox(dat)[0]
    elif transformation=='diff':
        dat = np.diff(dat)
    elif transformation=='diff_boxcox':
        dat = np.diff(boxcox(dat)[0])
    elif transformation=='diff_log':
        dat = np.diff(np.log(dat))
    elif transformation=='log':
        dat = np.log(dat)
    elif transformation=='none':
        dat = data
        
    if na_action:
        return na_omit(dat)
    else:
        return dat, lmbda

In [12]:
# for holdout forecasting & transformation selection

from scipy.stats import boxcox
from scipy.special import inv_boxcox

def na_omit(x):
    x = np.array(x)
    return x[~np.isnan(x)]

def transform(data,transformation,lmbda=None,na_action=False):
    dat = np.array(data)
    if transformation=='boxcox':
        dat = boxcox(dat,lmbda)
    elif transformation=='diff':
        dat = np.diff(dat)
    elif transformation=='diff_boxcox':
        dat = np.diff(boxcox(dat,lmbda))
    elif transformation=='diff_log':
        dat = np.diff(np.log(dat))
    elif transformation=='log':
        dat = np.log(dat)
    elif transformation=='none':
        dat = data
        
    if na_action:
        return na_omit(dat)
    else:
        return dat
    
def inv_transform(data,transformation,lmbda=None,base=None,omit_first=False):
    dat = np.array(data)
    if transformation=='boxcox':
        dat = inv_boxcox(dat,lmbda)
    elif transformation=='diff':
        dat = np.cumsum([base]+dat.tolist())
        if omit_first:
            dat = dat[1:]
    elif transformation=='diff_boxcox':
        dat = inv_boxcox(np.cumsum([boxcox(base,lmbda)]+dat.tolist()),lmbda)
        if omit_first:
            dat = dat[1:]
    elif transformation=='diff_log':
        dat = np.exp(np.cumsum(np.concatenate(([np.log(base)],dat))))
        if omit_first:
            dat = dat[1:]
    elif transformation=='log':
        dat = np.exp(dat)
    elif transformation=='none':
        dat = dat
    return dat
    
def evaluate_transformation_2(train,test,season):
    
    lmbda = boxcox(train)[1]
    base = np.flip(train,axis=0)[0]
    transformations = ["none","log","diff","diff_log","boxcox_","diff_boxcox"]
    mases = defaultdict(list)
    
    for t in transformations:
        ro.globalenv["tmp"] = transform(train,t,lmbda,base)
        try:
            if season == 1:
                fit = forecast.auto_arima(ro.globalenv["tmp"],
                                          seasonal=False,
                                          max_order=10,
                                          max_d=3,
                                          stepwise=True,
                                          parallel=True,
                                          num_cores=6)
            else:
                fit = forecast.auto_arima(ro.globalenv["tmp"],
                                          max_order=12, 
                                          start_p=0,
                                          start_q=0,
                                          max_d=3,
                                          max_D=2,
                                          stepwise=True,
                                          parallel=True,
                                          num_cores=6)
            
            preds_tmp = forecast.forecast(fit, h=test.shape[0])
            preds_treal = np.array(preds_tmp[3], dtype=np.float32)
            preds_treal = inv_transform(preds_treal, t, lmbda, base, True)
            mases[t].append(MASE(train, test, preds_treal, season))
        except:
            if t=="none":
                 mases[t].append(1)
            continue
    
    
    best = min(mases,key=mases.get)
    return best, lmbda, mases[best][0]

In [96]:
import warnings
warnings.filterwarnings('ignore')

def train_model_arima(data, filler_cols = (2,1), transform_ts=True, smooth=False, pre_calc_tr=False):
    '''
    Use all the data to train models for each row using given method
    
    Parameters:
        method: method used for model (i.e LinearRegression, MLPRegressor, RandomForestRegressor, etc)
        data: (dict) containing key: train, test
        filler_cols: (tuple) 2-element tuple containing number of filler cols for train and test
        multiplier: (int) how many seasons are taken into account for data sampling
        methods_kwargs: (dict) dictionary that contains keyword argument pairs for method using for training the model
        fit_kwargs: (dict) dictionary that contains keyword argument pairs for fitting method of the model.
        
    Returns:
        y_preds: (list) list containing all the predictions
    '''
    

    a = time.time()  # Starting time

    preds = defaultdict(list)
    mases = defaultdict(list)
    smapes = defaultdict(list)
    train_mases = defaultdict(dict)
    
    #######
    if transform_ts:
        transformations = defaultdict(dict)
        alpha = 0.1e-3
    #######

    for freq, (train, test) in data.items():
        
        if transform_ts:
            transformations[freq] = defaultdict(dict) 

        test = test.values[:, filler_cols[1]:]  # Drop filler cols
        season, horizon = season_horizon[freq]  # Get seasonality and horizon (frequence)
        
        a1 = time.time()  # Time end
        
        # Get predictions one by one
        for i, (ix, item) in enumerate(train.iterrows()):
            # specific stuff
            train_row = np.array(item[filler_cols[0]:].dropna().values, dtype=np.float32)
            test_row = np.array(test[ix], dtype=np.float32)
            
            
            ##########################
            ## Anomaly smoother
            ##########################
            
            if smooth:
                ts = pd.Series(train_row)
                rolling_avg = ts.rolling(30,min_periods=1,center=True).mean()
                rolling_std = ts.rolling(30,min_periods=1,center=True).std()
    
                for n,j in enumerate(ts.tolist()):
                    if np.absolute((j - rolling_avg[n]) / rolling_std[n]) >= 2:
                        train_row[n] = rolling_avg[n]

            ##########################
            ## Transformations
            ##########################
            
            best = "none"
            lmbda = 0
            train_mase = "none"
            if transform_ts:
                
                if train_row[:train_row.shape[0]-horizon].shape[0] > horizon:
                    best,lmbda,train_mase = evaluate_transformation_2(train_row[:train_row.shape[0]-horizon],
                                                                      train_row[train_row.shape[0]-horizon:],
                                                                      season)
                
                
                transformations[freq][item[0]] = {"best_transformation": best,
                                                  "lambda": lmbda,
                                                  "base_train": train_row[0],
                                                  "base_test": np.flip(train_row,axis=0)[0]}
                
                if best!="none":
                    train_row_old = train_row
                    train_row = transform(train_row, best, lmbda)
                    
            if pre_calc_tr:
                best = t_list.loc[t_list.label==item[0], "transformation"].tolist()[0]
                if best!="none":
                    train_row_old = train_row
                    train_row, lmbda = transform_2(train_row, best)
                        
            ##########################
            ##########################
            ##########################
            
            # Train model
            # https://www.rdocumentation.org/packages/forecast/versions/7.3/topics/auto.arima
            ro.globalenv["train"] = train_row
            #ro.globalenv["test"] = test_row
            fit = forecast.auto_arima(ro.globalenv["train"], max_order = 20, start_p = 0, start_q = 0)
            preds_tmp = forecast.forecast(fit, h=horizon)
            preds_treal = np.array(preds_tmp[3], dtype=np.float32)
            
            
            ##########################
            ## Inverse transformations
            ##########################
            
            if transform_ts: 
                if best!="none":
                    train_row = train_row_old
                    preds_treal = inv_transform(preds_treal, best, lmbda, transformations[freq][item[0]]["base_test"], True)
                    
            if pre_calc_tr:
                if best!="none":
                    train_row = train_row_old
                    preds_treal = inv_transform(preds_treal, best, lmbda, np.flip(train_row,axis=0)[0], True)
            
            ##########################
            ##########################
            ##########################
            
            preds[freq].append(preds_treal)
            mases[freq].append(MASE(train_row, test_row, preds_treal, season))
            smapes[freq].append(sMAPE(test_row, preds_treal))
            
            if np.isnan(mases[freq][len(mases[freq])-1]):
                print(item[0],best)
            
            train_mases[freq][item[0]] = {"MASE":train_mase,
                                          "transformation":best}

        b1 = time.time()  # Time end
        total1 = b1 - a1
        print("%s: %.3f" % (freq, total1))

    # Time 
    b = time.time()
    total = b - a
    print("Total time taken: ", total, "\n")  # How much time spent?

    return preds, mases, smapes, train_mases

In [ ]:
preds, mases, smapes, train_mases = train_model_arima(all_data, filler_cols=(2,1), transform_ts=True, smooth=False, pre_calc_tr=False)

In [63]:
ts_name = []
ts_tran = []
for i,row in enumerate(train_mases["monthly"]):
    ts_name.append(row)
    ts_tran.append(train_mases["monthly"][row]['transformation'])

t_list = pd.DataFrame({"label": ts_name, "transformation": ts_tran})
t_list.to_csv("monthly_transformations.csv",index=False)

In [ ]:
def export_preds(all_data, preds, outp_name):
    dfs = []
    
    pointer = 0

    for i, (freq, (train, test)) in enumerate(all_data.items()):
        len_t = len(train)
        temp_preds = preds[freq]
        df = pd.DataFrame(temp_preds)
        df.index = train['V1']
        dfs.append(df)
        pointer = len_t

    df_all = pd.concat(dfs)    
    df_all.to_csv(outp_name + ".csv", header=True)
export_preds(all_data, preds, "results/RandomF_arima_preprocessed_holdout")

In [ ]:
all_data = {"monthly": (pd.read_csv("data/full/Monthly-train.csv"), test_mo)}

In [97]:
preds, mases, smapes, train_mases = train_model_arima(all_data, filler_cols=(1,0), transform_ts=False, smooth=False, pre_calc_tr=True)

M2766 diff_boxcox
M2912 diff_boxcox
M3011 diff_boxcox
M7585 diff_boxcox
M7633 diff_boxcox
M8391 diff_boxcox
M8545 diff_boxcox
M9194 diff_boxcox
M9595 diff_boxcox
M9800 diff_boxcox
M15361 diff_boxcox
M20655 diff_boxcox
M28787 diff_boxcox
M33647 diff_boxcox
M34949 diff_boxcox
M36447 diff_boxcox
M38737 diff_boxcox
M39291 diff_boxcox
M39754 diff_boxcox
M40930 diff_boxcox
M41912 diff_boxcox
monthly: 3493.687
Total time taken:  3493.687147140503 



In [98]:
export_preds(all_data, preds, "results/RandomF_arima_preprocessed_test")